# Import

In [1]:
# https://www.kaggle.com/gogo827jz/jane-street-ffill-xgboost-purgedtimeseriescv

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import sys, inspect
import os, gc
import numpy as np
from numba import njit
import datatable as dtable
import pandas as pd
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tqdm.notebook import tqdm
from joblib import dump, load

In [5]:
TRAINING = True
ENSEMBLE = False
FOLDS = 4
SEED = 42
DATA_ROOT = '/storage1/lu/Active/tianyang/Workspace/janestreet/'
DATA_FILE = DATA_ROOT + 'train.feather'  # feather is faster than csv

# janestreet
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
libdir = os.path.join(currentdir, '..', 'data')
sys.path.insert(0, libdir) 

# Train

In [6]:
print('Loading...')
train = pd.read_feather(DATA_FILE)
features = [c for c in train.columns if 'feature' in c]

print('Filling...')
train = train.query('weight > 0').reset_index(drop = True)
train[features] = train[features].fillna(method = 'ffill').fillna(0)
train['action'] = (train['resp'] > 0).astype('int')

print('Finish.')

X_tr, y_tr = train.loc[train['date'] > 85, features].values, train.loc[train['date'] > 85, 'action'].values
d_tr = xgb.DMatrix(X_tr, y_tr)

Loading...
Filling...
Finish.


In [7]:
%%time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBClassifier

# Seed Blending
models = []

p_best = {
    'learning_rate': 0.014106988708201764,
    'max_depth': 8, 
    'gamma': 9.800749651802157, 
    'min_child_weight': 0.3032862674190433, 
    'subsample': 0.4648851101943981, 
    'colsample_bytree': 0.994909039539885, 
    'objective': 'binary:logistic',
    'eval_metric': 'auc', 
    'tree_method': 'hist', 
 }
    
if TRAINING:
    # scores = cross_validate(clf, X, yy, scoring='roc_auc', cv=cv, n_jobs=-1, verbose=10)
    
    if ENSEMBLE:
        for seed in range(5):
            p_best['random_state'] = seed
            clf = xgb.train(p_best, d_tr, 950)
            models.append(clf)

            rubbish = gc.collect()
    else:
        clf = xgb.train(p_best, d_tr, 950)
        models = [clf]

    dump(clf, 'xgb.joblib')
    
else:
    clf = load('xgb.joblib')
    models = [clf]
    

CPU times: user 5h 47min 35s, sys: 11min 22s, total: 5h 58min 58s
Wall time: 22min 37s


# Predict

In [8]:
@njit
def fast_fillna(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [9]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

opt_th = 0.505
tmp = np.zeros(len(features))
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        x_tt[0, :] = fast_fillna(x_tt[0, :], tmp)
        tmp = x_tt[0, :]
        d_tt = xgb.DMatrix(x_tt)
        pred = 0.
        for clf in models:
            pred += clf.predict(d_tt) / len(models)
        pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

|          | 0/? [00:00<?, ?it/s]

In [10]:
pred = pd.read_csv('submission.csv')
pred['action'].sum()

6143

In [11]:
len(pred['action'])

15219